# Dichiarazione di funzioni necessarie a risolvere istanze di un problema di binary paintshop #

In [ ]:
import numpy as np
from itertools import product
import copy
import matplotlib.pyplot as plt
from scipy import optimize

from qiskit.algorithms import QAOA, NumPyMinimumEigensolver
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import Aer, execute
from qiskit.utils import algorithm_globals, QuantumInstance
from qiskit.circuit import Parameter
from qiskit_optimization.algorithms import MinimumEigenOptimizer, RecursiveMinimumEigenOptimizer
from qiskit_optimization import QuadraticProgram
from qiskit import IBMQ
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer import QasmSimulator
from qiskit.test.mock import FakeMontreal

from dimod.utilities import ising_to_qubo
from dimod.reference.samplers import ExactSolver
from dwave.system import DWaveSampler, EmbeddingComposite

In [ ]:
def color_changes(paint_bitstring):
    color_sequence = []
    painted_once = set()
    for car in CAR_SEQUENCE:
        if car in painted_once:
            color_sequence.append(not paint_bitstring[car])
        else:
            color_sequence.append(paint_bitstring[car])
            painted_once.add(car)
    paint_change_counter = 0
    for color0, color1 in zip(color_sequence[:-1], color_sequence[1:]):
        if color0 != color1:
            paint_change_counter += 1
    return paint_change_counter

In [ ]:
def init_coeff():
    same = -1
    different = 1
    appeared_already = set()
    for car0, car1 in zip(CAR_SEQUENCE[:-1], CAR_SEQUENCE[1:]):
        if car0 == car1:
            continue
        if car0 in appeared_already:
            appeared_already.add(car0)
            if car1 in appeared_already:
                yield [(car0, car1), same]
            else:
                yield [(car0, car1), different]
        else:
            appeared_already.add(car0)
            if car1 in appeared_already:
                yield [(car0, car1), different]
            else:
                yield [(car0, car1), same]

In [ ]:
def coeff_to_dict(lista):
    quad = dict()
    for item in lista:
        if (item[0][0], item[0][1]) in quad.keys():
            quad[item[0][0], item[0][1]] += item[1]
        elif (item[0][1], item[0][0]) in quad:
            quad[item[0][1], item[0][0]] += item[1]
        else:
            quad[item[0][0], item[0][1]] = item[1]
    return quad

In [ ]:
def color_changes_from_initial(initial, echo=False):
    counter = 0
    dynamic_sequence = initial
    dynamic_sequence
    
    for i in range(CAR_PAIR_COUNT*2 - 1):
        if (dynamic_sequence[CAR_SEQUENCE[i]] != dynamic_sequence[CAR_SEQUENCE[i + 1]])\
            or\
            (CAR_SEQUENCE[i] == CAR_SEQUENCE[i + 1]):
            counter += 1
        if echo:
            print(i,dynamic_sequence[CAR_SEQUENCE[i]], CAR_SEQUENCE[i])
        dynamic_sequence[CAR_SEQUENCE[i]] =  int((dynamic_sequence[CAR_SEQUENCE[i]]) ^ True)
        #print(CAR_SEQUENCE[i], CAR_SEQUENCE[i+1], dynamic_sequence, counter)
    return counter

QAOA:

In [ ]:
def exec_qaoa(p, noisy=False):
    noise_model = NoiseModel.from_backend(FakeMontreal())
#     algorithm_globals.random_seed = 42
    algorithm_globals.massive=True
    
    if noisy:
        noise_model = NoiseModel.from_backend(FakeMontreal())
        
        backend = Aer.get_backend('qasm_simulator')
#         quantum_instance = QuantumInstance(backend,
#                                        seed_simulator=algorithm_globals.random_seed,
#                                        seed_transpiler=algorithm_globals.random_seed,
#                                        noise_model=noise_model)
        quantum_instance = QuantumInstance(backend,
                                       seed_simulator=algorithm_globals.random_seed,
                                       seed_transpiler=algorithm_globals.random_seed)
    
        qaoa_mes = QAOA(quantum_instance=quantum_instance, reps=p, include_custom = True)
        qaoa_noisy = MinimumEigenOptimizer(qaoa_mes)
        return qaoa_noisy.solve(qubo)
    else:
        backend = QasmSimulator(method='statevector')
        quantum_instance = QuantumInstance(backend,
                                           seed_simulator=algorithm_globals.random_seed,
                                           seed_transpiler=algorithm_globals.random_seed)
        qaoa_mes = QAOA(quantum_instance=quantum_instance, reps=p, include_custom = True)
        qaoa_noiseless = MinimumEigenOptimizer(qaoa_mes)   # using QAOA
        return qaoa_noiseless.solve(qubo)

Greedy:

In [ ]:
def color_changes_greedy(paint_bitstring, echo=False):
    paint_zero= set()
    paint_one = set()
    status = 0
    counter = 0
    path = []
    result = np.zeros(len(paint_bitstring) // 2)
    for i in range(len(paint_bitstring)):
        path.append(status)
        if (paint_bitstring[i] not in paint_zero) and (paint_bitstring[i] not in paint_one):
            result[paint_bitstring[i]] = status
            
        if status == 0 and (paint_bitstring[i] not in paint_zero):
            paint_zero.add(paint_bitstring[i])
        elif status == 0 and (paint_bitstring[i] in paint_zero):
            counter += 1
            status = 1
            paint_one.add(paint_bitstring[i])
            
        elif status == 1 and (paint_bitstring[i] in paint_one):
            counter += 1
            status = 0
            paint_zero.add(paint_bitstring[i])
            
        elif status == 1 and (paint_bitstring[i] not in paint_one):
            paint_one.add(paint_bitstring[i])
        if echo:
            print(i,status, CAR_SEQUENCE[i])
    
    return list(map(int, result)), counter

NumPy:

In [ ]:
def exec_np():
    exact_mes = NumPyMinimumEigensolver()
    exact = MinimumEigenOptimizer(exact_mes)  # using the exact classical numpy minimum eigen solver
    return exact.solve(qubo)

Casuale:

In [ ]:
def color_changes_random(paint_bitstring, echo=False):
    paint_zero= set()
    paint_one = set()
    status = np.random.randint(0,2)
    counter = 0
    path = []
    result = np.zeros(len(paint_bitstring) // 2)
    for i in range(len(paint_bitstring)):
        path.append(status)
        if (paint_bitstring[i] not in paint_zero) and (paint_bitstring[i] not in paint_one):
            result[paint_bitstring[i]] = np.random.randint(0,2)
            
        if status == 0 and (paint_bitstring[i] not in paint_zero):
            paint_zero.add(paint_bitstring[i])
        elif status == 0 and (paint_bitstring[i] in paint_zero):
            counter += 1
            status = 1
            paint_one.add(paint_bitstring[i])
            
        elif status == 1 and (paint_bitstring[i] in paint_one):
            counter += 1
            status = 0
            paint_zero.add(paint_bitstring[i])
            
        elif status == 1 and (paint_bitstring[i] not in paint_one):
            paint_one.add(paint_bitstring[i])
        if echo:
            print(i,status, CAR_SEQUENCE[i])
    
    return list(map(int, result)), counter

# Creazione dati iniziali #

In [ ]:
CAR_PAIR_COUNT = 5
CAR_SEQUENCE = np.random.permutation([x for x in range(CAR_PAIR_COUNT)] * 2)

CAR_SEQUENCE

In [ ]:
linear = [0]*CAR_PAIR_COUNT

quadratic = coeff_to_dict(list(init_coeff()))

J = quadratic
h = {}
qubo_temp, _ = ising_to_qubo(h, J)

In [ ]:
linear_qubo = {(i1):qubo_temp[i1, i2]  for i1, i2 in qubo_temp.keys() if i1 == i2}
quadratic_qubo = {(i1, i2):qubo_temp[i1, i2]  for i1, i2 in qubo_temp.keys() if i1 != i2}

Creazione oggetto _QuadraticProgram_:

In [ ]:
qubo = QuadraticProgram()
for i in range(CAR_PAIR_COUNT):    
    qubo.binary_var(f"car_{str(i)}")

qubo.minimize(linear=linear_qubo, quadratic=quadratic_qubo)

# Risoluzione del problema #

## QAOA: ##

In [ ]:
max_p = 4

best_result_qaoa_noisy = np.zeros((max_p+1, CAR_PAIR_COUNT), dtype=int)
best_result_qaoa_noiseless = np.zeros((max_p+1, CAR_PAIR_COUNT), dtype=int)

for p in range(1, max_p+1):
    print(f"Evaluating P={p} noisy")
    best_result_qaoa_noisy[p] = list(map(int, exec_qaoa(p, True).x))
    print(f"Evaluating P={p} noise-less")
    best_result_qaoa_noiseless[p] = list(map(int, exec_qaoa(p, False).x))




## NumPy: ##

In [ ]:
best_result_np = list(map(int, exec_np().x))

## QA: ##

In [ ]:
qa_sim = True            #use a simulator if True
if qa_sim:
    sampler = ExactSolver()
    sample = sampler.sample_ising(h, J)
else:
    from dwave.system import LeapHybridSampler
    sampler_auto = LeapHybridSampler(solver={'category': 'hybrid'})
    sample = sampler_auto.sample_ising(h, J)

best_result_qa = list(map(lambda x : int(x>0), list(sample.first.sample.values())))

# sample.to_pandas_dataframe().sort_values("energy")

## Greedy: ##

In [ ]:
result_greedy, CC_greedy = color_changes_greedy(CAR_SEQUENCE, echo=False)

# Salvataggio dei risultati in memoria secondaria: #

In [ ]:
random_r = color_changes_random(CAR_SEQUENCE)
CC_random = color_changes_from_initial(random_r[0])


reference_res = color_changes(CAR_SEQUENCE)
random_res = CC_random
greedy_res = color_changes_from_initial(result_greedy.copy())

qa_res = color_changes_from_initial(best_result_qa.copy())
np_res = color_changes_from_initial(best_result_np.copy())

In [ ]:
print(f"Number of car pairs:\t{CAR_PAIR_COUNT}")
print(f"Car sequence:\t\t{CAR_SEQUENCE}\n")

print(f"Random Solution:\t\t{random_r[0]}\t Color changes:{random_res}")
print(f"Greedy Solution:\t\t{result_greedy}\t Color changes:{greedy_res}")

for p in range(1, max_p + 1):
    print(f"QAOA Noisy \tp={p} Solution:\t{list(best_result_qaoa_noisy[p])}"\
        f"\t Color changes:{color_changes_from_initial(best_result_qaoa_noisy[p].copy())}")
    print(f"QAOA Noise-Less p={p} Solution:\t{list(best_result_qaoa_noiseless[p])}"\
        f"\t Color changes:{color_changes_from_initial(best_result_qaoa_noiseless[p].copy())}")
print(f"QA   Solution:  \t\t{best_result_qa}\t Color changes:{qa_res}")
print(f"NumPy   Solution:  \t\t{best_result_np}\t Color changes:{np_res}")

In [ ]:
from pandas import DataFrame
to_print = DataFrame({"Sequence": [str(CAR_SEQUENCE)], "Random":random_res,\
                     "Greedy": greedy_res, "NumPy": np_res})
for p in range(1, max_p + 1):
    to_print[f"QAOA_p{p}_noisy"] = color_changes_from_initial(best_result_qaoa_noisy[p].copy())
    to_print[f"QAOA_p{p}_noiseless"] = color_changes_from_initial(best_result_qaoa_noiseless[p].copy())
to_print["QA"] = qa_res
to_print

In [ ]:
filename = f'values_{CAR_PAIR_COUNT}.csv'
import os

if os.path.exists(filename):
    header = False
else:
    header = True

with open(filename, 'a') as f:
    to_print.to_csv(filename, header=header, mode='a', index=False)
